<a href="https://colab.research.google.com/github/Abhilash0708/spark/blob/master/PySpark_by_Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Download Chicago's Reported Crime Data

## Download and install Spark

In [9]:
!ls

reported-crimes.csv  spark-2.3.1-bin-hadoop2.7	    spark-warehouse
sample_data	     spark-2.3.1-bin-hadoop2.7.tgz


In [10]:
#!apt-get update
#!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#!wget -q http://archive.apache.org/dist/spark/spark-2.3.1/spark-2.3.1-bin-hadoop2.7.tgz
#!tar xf spark-2.3.1-bin-hadoop2.7.tgz
#!pip install -q findspark

In [11]:
! ls

reported-crimes.csv  spark-2.3.1-bin-hadoop2.7	    spark-warehouse
sample_data	     spark-2.3.1-bin-hadoop2.7.tgz


## Setup environment

In [12]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.3.1-bin-hadoop2.7"

import findspark
findspark.init()
from pyspark import SparkContext
sc = SparkContext.getOrCreate()

import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate() 
spark

## Downloading and preprocessing Chicago's Reported Crime Data

In [13]:
#!wget https://data.cityofchicago.org/api/views/ijzp-q8t2/rows.csv?accessType=DOWNLOAD

In [14]:
# Listing Directory
! ls

reported-crimes.csv  spark-2.3.1-bin-hadoop2.7	    spark-warehouse
sample_data	     spark-2.3.1-bin-hadoop2.7.tgz


In [15]:
# Renaming File
#!mv rows.csv?accessType=DOWNLOAD reported-crimes.csv

In [16]:
from pyspark.sql.functions import to_timestamp,col,lit
rc = spark.read.csv('reported-crimes.csv',header=True).withColumn('Date',to_timestamp(col('Date'),'MM/dd/yyyy hh:mm:ss a')).filter(col('Date') <= lit('2018-11-11'))
#100 years completion on 11th November 2018.
rc.show(10)

+--------+-----------+-------------------+--------------------+----+------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|      ID|Case Number|               Date|               Block|IUCR|      Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|    Latitude|    Longitude|            Location|
+--------+-----------+-------------------+--------------------+----+------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|10224738|   HY411648|2015-09-05 13:30:00|     043XX S WOOD ST|0486|           BATTERY|DOMESTIC BATTERY ...|           RESIDENCE| fal

# Schemas

In [17]:
rc.printSchema()

root
 |-- ID: string (nullable = true)
 |-- Case Number: string (nullable = true)
 |-- Date: timestamp (nullable = true)
 |-- Block: string (nullable = true)
 |-- IUCR: string (nullable = true)
 |-- Primary Type: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Location Description: string (nullable = true)
 |-- Arrest: string (nullable = true)
 |-- Domestic: string (nullable = true)
 |-- Beat: string (nullable = true)
 |-- District: string (nullable = true)
 |-- Ward: string (nullable = true)
 |-- Community Area: string (nullable = true)
 |-- FBI Code: string (nullable = true)
 |-- X Coordinate: string (nullable = true)
 |-- Y Coordinate: string (nullable = true)
 |-- Year: string (nullable = true)
 |-- Updated On: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Longitude: string (nullable = true)
 |-- Location: string (nullable = true)



In [22]:
from pyspark.sql.types import StructType, StructField, StringType, TimestampType, BooleanType, DoubleType, IntegerType

In [23]:
rc.columns

['ID',
 'Case Number',
 'Date',
 'Block',
 'IUCR',
 'Primary Type',
 'Description',
 'Location Description',
 'Arrest',
 'Domestic',
 'Beat',
 'District',
 'Ward',
 'Community Area',
 'FBI Code',
 'X Coordinate',
 'Y Coordinate',
 'Year',
 'Updated On',
 'Latitude',
 'Longitude',
 'Location']

In [24]:
labels = [
   ( 'ID', StringType()),
   ('Case Number', StringType()),
   ('Date', TimestampType()),
   ('Block',  StringType()),
   ('IUCR', StringType()),
   ('Primary Type', StringType()),
   ('Description', StringType()),
   ('Location Description', StringType()),
   ('Arrest', StringType()),
   ('Domestic', BooleanType()),
   ('Beat', StringType()),
   ('District', StringType()),
   ('Ward', StringType()),
   ('Community Area', StringType()),
   ('FBI Code', StringType()),
   ('X Coordinate', StringType()),
   ('Y Coordinate', StringType()),
   ('Year', IntegerType()),
   ('Updated On', StringType()),
   ('Latitude', DoubleType()),
   ('Longitude', DoubleType()),
   ('Location', StringType())
]

In [25]:
schema = StructType([StructField (x[0], x[1], True) for x in labels])
schema

StructType(List(StructField(ID,StringType,true),StructField(Case Number,StringType,true),StructField(Date,TimestampType,true),StructField(Block,StringType,true),StructField(IUCR,StringType,true),StructField(Primary Type,StringType,true),StructField(Description,StringType,true),StructField(Location Description,StringType,true),StructField(Arrest,StringType,true),StructField(Domestic,BooleanType,true),StructField(Beat,StringType,true),StructField(District,StringType,true),StructField(Ward,StringType,true),StructField(Community Area,StringType,true),StructField(FBI Code,StringType,true),StructField(X Coordinate,StringType,true),StructField(Y Coordinate,StringType,true),StructField(Year,IntegerType,true),StructField(Updated On,StringType,true),StructField(Latitude,DoubleType,true),StructField(Longitude,DoubleType,true),StructField(Location,StringType,true)))

In [27]:
!ls

reported-crimes.csv  spark-2.3.1-bin-hadoop2.7	    spark-warehouse
sample_data	     spark-2.3.1-bin-hadoop2.7.tgz


In [28]:
rc =  spark.read.csv('reported-crimes.csv', schema = schema)

In [29]:
rc.printSchema()

root
 |-- ID: string (nullable = true)
 |-- Case Number: string (nullable = true)
 |-- Date: timestamp (nullable = true)
 |-- Block: string (nullable = true)
 |-- IUCR: string (nullable = true)
 |-- Primary Type: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Location Description: string (nullable = true)
 |-- Arrest: string (nullable = true)
 |-- Domestic: boolean (nullable = true)
 |-- Beat: string (nullable = true)
 |-- District: string (nullable = true)
 |-- Ward: string (nullable = true)
 |-- Community Area: string (nullable = true)
 |-- FBI Code: string (nullable = true)
 |-- X Coordinate: string (nullable = true)
 |-- Y Coordinate: string (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Updated On: string (nullable = true)
 |-- Latitude: double (nullable = true)
 |-- Longitude: double (nullable = true)
 |-- Location: string (nullable = true)



In [32]:
# Can't infer the Schema that we have provided. Problems with the data
rc.show(10)

+----+-----------+----+-----+----+------------+-----------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+----------+--------+---------+--------+
|  ID|Case Number|Date|Block|IUCR|Primary Type|Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|Updated On|Latitude|Longitude|Location|
+----+-----------+----+-----+----+------------+-----------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+----------+--------+---------+--------+
|null|       null|null| null|null|        null|       null|                null|  null|    null|null|    null|null|          null|    null|        null|        null|null|      null|    null|     null|    null|
|null|       null|null| null|null|        null|       null|                null|  null|    null|null|    null|null|          null|    null|        null|        

In [33]:
#df.column1 - Attribute
#df['column1'] - Indexing

In [34]:
!ls

reported-crimes.csv  spark-2.3.1-bin-hadoop2.7	    spark-warehouse
sample_data	     spark-2.3.1-bin-hadoop2.7.tgz


# Using Inferschema and Header to read data

In [40]:
rc =  spark.read.csv('reported-crimes.csv', inferSchema=True, header=True)

# Working with Columns

## Display only the first 5 rows in column name IUCR

In [41]:
rc.show(5)

+--------+-----------+--------------------+--------------------+----+------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|      ID|Case Number|                Date|               Block|IUCR|Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|    Latitude|    Longitude|            Location|
+--------+-----------+--------------------+--------------------+----+------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|10224738|   HY411648|09/05/2015 01:30:...|     043XX S WOOD ST|0486|     BATTERY|DOMESTIC BATTERY ...|           RESIDENCE| false|    true| 924|   

In [42]:
rc.select('IUCR').show(5)

+----+
|IUCR|
+----+
|0486|
|0870|
|0810|
|2023|
|0560|
+----+
only showing top 5 rows



In [43]:
rc.select(rc.IUCR).show(5)

+----+
|IUCR|
+----+
|0486|
|0870|
|0810|
|2023|
|0560|
+----+
only showing top 5 rows



In [44]:
rc.select(col('IUCR')).show(5)

+----+
|IUCR|
+----+
|0486|
|0870|
|0810|
|2023|
|0560|
+----+
only showing top 5 rows



## Displaying only the first 4 rows of the column names Case Number, Date and Arrest

In [45]:
rc.select('Case Number', 'Date', 'Arrest').show(4)

+-----------+--------------------+------+
|Case Number|                Date|Arrest|
+-----------+--------------------+------+
|   HY411648|09/05/2015 01:30:...| false|
|   HY411615|09/04/2015 11:30:...| false|
|   JC213529|09/01/2018 12:01:...| false|
|   HY411595|09/05/2015 12:45:...|  true|
+-----------+--------------------+------+
only showing top 4 rows



## Add a column name One with all entries as 1

In [46]:
from pyspark.sql.functions import lit

In [48]:
rc.withColumn('One', lit(1)).show(5)

+--------+-----------+--------------------+--------------------+----+------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+---+
|      ID|Case Number|                Date|               Block|IUCR|Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|    Latitude|    Longitude|            Location|One|
+--------+-----------+--------------------+--------------------+----+------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+---+
|10224738|   HY411648|09/05/2015 01:30:...|     043XX S WOOD ST|0486|     BATTERY|DOMESTIC BATTERY ...|           RESIDENCE| false|    t

## Remove Column IUCR

In [49]:
rc = rc.drop('IUCR')
rc.show(5)

+--------+-----------+--------------------+--------------------+------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|      ID|Case Number|                Date|               Block|Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|    Latitude|    Longitude|            Location|
+--------+-----------+--------------------+--------------------+------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|10224738|   HY411648|09/05/2015 01:30:...|     043XX S WOOD ST|     BATTERY|DOMESTIC BATTERY ...|           RESIDENCE| false|    true| 924|       9|  12|         

# Working with Rows

In [50]:
!ls

reported-crimes.csv  spark-2.3.1-bin-hadoop2.7	    spark-warehouse
sample_data	     spark-2.3.1-bin-hadoop2.7.tgz


In [51]:
rc =  spark.read.csv('reported-crimes.csv', inferSchema=True, header=True)

## Add the reported crimes for an additional day say, 12th November 2018

In [56]:
one_day =  spark.read.csv('reported-crimes.csv',header=True, inferSchema = True).withColumn('Date',to_timestamp(col('Date'),'MM/dd/yyyy hh:mm:ss a')).filter(col('Date') == lit('2018-11-12'))
one_day.count()

3

In [57]:
one_day.show()

+--------+-----------+-------------------+-------------------+----+------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|      ID|Case Number|               Date|              Block|IUCR|      Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|    Latitude|    Longitude|            Location|
+--------+-----------+-------------------+-------------------+----+------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|11505149|   JB513151|2018-11-12 00:00:00| 003XX S WHIPPLE ST|0810|             THEFT|           OVER $500|              STREET| false| 

In [59]:
#Appending the data
rc.union(one_day).orderBy('Date', ascending = False).show(5)

+--------+-----------+--------------------+-------------------+----+------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|      ID|Case Number|                Date|              Block|IUCR|      Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|    Latitude|    Longitude|            Location|
+--------+-----------+--------------------+-------------------+----+------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|11505149|   JB513151| 2018-11-12 00:00:00| 003XX S WHIPPLE ST|0810|             THEFT|           OVER $500|              STREET| fal

## What is the top 10 number of reported crimes by Primary Type, in descending order of occurence?

In [66]:
rc.groupby('Primary Type').count().show(truncate = False)

+--------------------------------+-------+
|Primary Type                    |count  |
+--------------------------------+-------+
|OFFENSE INVOLVING CHILDREN      |54598  |
|CRIMINAL SEXUAL ASSAULT         |5801   |
|STALKING                        |4538   |
|PUBLIC PEACE VIOLATION          |51948  |
|OBSCENITY                       |790    |
|NON-CRIMINAL (SUBJECT SPECIFIED)|9      |
|ARSON                           |13040  |
|DOMESTIC VIOLENCE               |1      |
|GAMBLING                        |14616  |
|CRIMINAL TRESPASS               |212030 |
|ASSAULT                         |496987 |
|NON - CRIMINAL                  |38     |
|LIQUOR LAW VIOLATION            |14804  |
|MOTOR VEHICLE THEFT             |359402 |
|THEFT                           |1615043|
|BATTERY                         |1403019|
|ROBBERY                         |287537 |
|HOMICIDE                        |12234  |
|RITUALISM                       |24     |
|PUBLIC INDECENCY                |192    |
+----------

In [67]:
spark = SparkSession.builder.config("spark.sql.repl.eagerEval.enabled", True).getOrCreate()

In [69]:
rc.groupby('Primary Type').count().show(n=5, truncate=False, vertical=True)


-RECORD 0----------------------------------
 Primary Type | OFFENSE INVOLVING CHILDREN 
 count        | 54598                      
-RECORD 1----------------------------------
 Primary Type | CRIMINAL SEXUAL ASSAULT    
 count        | 5801                       
-RECORD 2----------------------------------
 Primary Type | STALKING                   
 count        | 4538                       
-RECORD 3----------------------------------
 Primary Type | PUBLIC PEACE VIOLATION     
 count        | 51948                      
-RECORD 4----------------------------------
 Primary Type | OBSCENITY                  
 count        | 790                        
only showing top 5 rows



In [71]:
rc.groupby('Primary Type').count().show(n=5, truncate=True, vertical=False)


+--------------------+-----+
|        Primary Type|count|
+--------------------+-----+
|OFFENSE INVOLVING...|54598|
|CRIMINAL SEXUAL A...| 5801|
|            STALKING| 4538|
|PUBLIC PEACE VIOL...|51948|
|           OBSCENITY|  790|
+--------------------+-----+
only showing top 5 rows



In [72]:
%%html
<style>
div.output_area pre {
    white-space: pre;
}
</style>

In [76]:
import pandas as pd

In [78]:
rc.groupBy('Primary Type').count().orderBy('count', ascending = False).show()

+--------------------+-------+
|        Primary Type|  count|
+--------------------+-------+
|               THEFT|1615043|
|             BATTERY|1403019|
|     CRIMINAL DAMAGE| 872873|
|           NARCOTICS| 744296|
|             ASSAULT| 496987|
|       OTHER OFFENSE| 476086|
|            BURGLARY| 420525|
| MOTOR VEHICLE THEFT| 359402|
|  DECEPTIVE PRACTICE| 335241|
|             ROBBERY| 287537|
|   CRIMINAL TRESPASS| 212030|
|   WEAPONS VIOLATION| 102378|
|        PROSTITUTION|  69722|
|OFFENSE INVOLVING...|  54598|
|PUBLIC PEACE VIOL...|  51948|
|         SEX OFFENSE|  30010|
| CRIM SEXUAL ASSAULT|  27700|
|INTERFERENCE WITH...|  18148|
|LIQUOR LAW VIOLATION|  14804|
|            GAMBLING|  14616|
+--------------------+-------+
only showing top 20 rows



In [80]:

rc.groupBy('Primary Type').count().orderBy('count', ascending = False).limit(15).toPandas()

,Primary Type,count
0,THEFT,1615043
1,BATTERY,1403019
2,CRIMINAL DAMAGE,872873
3,NARCOTICS,744296
4,ASSAULT,496987
5,OTHER OFFENSE,476086
6,BURGLARY,420525
7,MOTOR VEHICLE THEFT,359402
8,DECEPTIVE PRACTICE,335241
9,ROBBERY,287537


In [81]:
from IPython.core.display import HTML

HTML("""<style>
    .output-plaintext, .output-stream, .output{
        white-space: pre !important;
        font-family: Monaco; # Any monospaced font should work
    }</style>""")

In [84]:
rc.groupBy('Primary Type').count().orderBy('count', ascending = False).show(truncate = False)

+--------------------------------+-------+
|Primary Type                    |count  |
+--------------------------------+-------+
|THEFT                           |1615043|
|BATTERY                         |1403019|
|CRIMINAL DAMAGE                 |872873 |
|NARCOTICS                       |744296 |
|ASSAULT                         |496987 |
|OTHER OFFENSE                   |476086 |
|BURGLARY                        |420525 |
|MOTOR VEHICLE THEFT             |359402 |
|DECEPTIVE PRACTICE              |335241 |
|ROBBERY                         |287537 |
|CRIMINAL TRESPASS               |212030 |
|WEAPONS VIOLATION               |102378 |
|PROSTITUTION                    |69722  |
|OFFENSE INVOLVING CHILDREN      |54598  |
|PUBLIC PEACE VIOLATION          |51948  |
|SEX OFFENSE                     |30010  |
|CRIM SEXUAL ASSAULT             |27700  |
|INTERFERENCE WITH PUBLIC OFFICER|18148  |
|LIQUOR LAW VIOLATION            |14804  |
|GAMBLING                        |14616  |
+----------

In [85]:
!ls

reported-crimes.csv  spark-2.3.1-bin-hadoop2.7	    spark-warehouse
sample_data	     spark-2.3.1-bin-hadoop2.7.tgz


## What % of reported crime resulted in an arrest?

In [86]:
rc.select('Arrest').distinct().show()

+------+
|Arrest|
+------+
|  true|
| false|
+------+



In [87]:
rc.printSchema()

root
 |-- ID: integer (nullable = true)
 |-- Case Number: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Block: string (nullable = true)
 |-- IUCR: string (nullable = true)
 |-- Primary Type: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Location Description: string (nullable = true)
 |-- Arrest: boolean (nullable = true)
 |-- Domestic: boolean (nullable = true)
 |-- Beat: integer (nullable = true)
 |-- District: integer (nullable = true)
 |-- Ward: integer (nullable = true)
 |-- Community Area: integer (nullable = true)
 |-- FBI Code: string (nullable = true)
 |-- X Coordinate: integer (nullable = true)
 |-- Y Coordinate: integer (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Updated On: string (nullable = true)
 |-- Latitude: double (nullable = true)
 |-- Longitude: double (nullable = true)
 |-- Location: string (nullable = true)



In [91]:
# What % of reported crimes resulted in an arrest?
print("The % of reported crimes that resulted in an arrest is?")
print(100 * rc.filter(col('Arrest') == 'true').count() / rc.select('Arrest').count(), "%")

The % of reported crimes that resulted in an arrest is?
26.372463422677452 %


## Top 3 locations for reported crimes?

In [97]:
rc.groupBy('Location Description').count().orderBy('count', ascending = False).limit(3).toPandas()

,Location Description,count
0,STREET,1988722
1,RESIDENCE,1289590
2,APARTMENT,855302


## Built-in Functions

In [98]:
from pyspark.sql import functions

In [99]:
print(dir(functions))

['AutoBatchedSerializer', 'Column', 'DataFrame', 'DataType', 'PandasUDFType', 'PickleSerializer', 'PythonEvalType', 'SparkContext', 'StringType', 'UserDefinedFunction', '__all__', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', '_binary_mathfunctions', '_collect_list_doc', '_collect_set_doc', '_create_binary_mathfunction', '_create_function', '_create_udf', '_create_window_function', '_functions', '_functions_1_4', '_functions_1_6', '_functions_2_1', '_functions_deprecated', '_lit_doc', '_message', '_string_functions', '_test', '_to_java_column', '_to_seq', '_window_functions', '_wrap_deprecated_function', 'abs', 'acos', 'add_months', 'approxCountDistinct', 'approx_count_distinct', 'array', 'array_contains', 'asc', 'ascii', 'asin', 'atan', 'atan2', 'avg', 'base64', 'bin', 'bitwiseNOT', 'blacklist', 'broadcast', 'bround', 'cbrt', 'ceil', 'coalesce', 'col', 'collect_list', 'collect_set', 'column', 'concat', 'concat_ws', 'conv', 'c

## String Functions

### Display the primary type column in lower and upper character, and the first 4 characters of the column

In [100]:
from pyspark.sql.functions import lower, upper, substring

In [101]:
help(substring)

Help on function substring in module pyspark.sql.functions:

substring(str, pos, len)
    Substring starts at `pos` and is of length `len` when str is String type or
    returns the slice of byte array that starts at `pos` in byte and is of length `len`
    when str is Binary type.
    
    .. note:: The position is not zero based, but 1 based index.
    
    >>> df = spark.createDataFrame([('abcd',)], ['s',])
    >>> df.select(substring(df.s, 1, 2).alias('s')).collect()
    [Row(s='ab')]
    
    .. versionadded:: 1.5



In [102]:
rc.printSchema()

root
 |-- ID: integer (nullable = true)
 |-- Case Number: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Block: string (nullable = true)
 |-- IUCR: string (nullable = true)
 |-- Primary Type: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Location Description: string (nullable = true)
 |-- Arrest: boolean (nullable = true)
 |-- Domestic: boolean (nullable = true)
 |-- Beat: integer (nullable = true)
 |-- District: integer (nullable = true)
 |-- Ward: integer (nullable = true)
 |-- Community Area: integer (nullable = true)
 |-- FBI Code: string (nullable = true)
 |-- X Coordinate: integer (nullable = true)
 |-- Y Coordinate: integer (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Updated On: string (nullable = true)
 |-- Latitude: double (nullable = true)
 |-- Longitude: double (nullable = true)
 |-- Location: string (nullable = true)



In [110]:
rc.select(lower(col('Primary Type')), upper(col('Primary Type')), substring(rc['Primary Type'],1,4)).limit(5).toPandas()

,lower(Primary Type),upper(Primary Type),"substring(Primary Type, 1, 4)"
0,battery,BATTERY,BATT
1,theft,THEFT,THEF
2,theft,THEFT,THEF
3,narcotics,NARCOTICS,NARC
4,assault,ASSAULT,ASSA


## Numeric Functions

### Show oldest and latest date

In [127]:
from pyspark.sql.types import *
rc.withColumn("DateTmp",rc.Date.cast(DateType())).show(5)

+--------+-----------+--------------------+--------------------+----+------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+-------+
|      ID|Case Number|                Date|               Block|IUCR|Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|    Latitude|    Longitude|            Location|DateTmp|
+--------+-----------+--------------------+--------------------+----+------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+-------+
|10224738|   HY411648|09/05/2015 01:30:...|     043XX S WOOD ST|0486|     BATTERY|DOMESTIC BATTERY ...|           RESIDENCE|

In [133]:
rc.select(col('Date')).distinct().show(truncate = False,ascending = True)

+----------------------+
|Date                  |
+----------------------+
|09/05/2015 08:06:00 PM|
|09/06/2015 12:24:00 AM|
|09/06/2015 04:06:00 AM|
|09/06/2015 12:15:00 PM|
|09/07/2015 09:00:00 AM|
|07/07/2015 08:00:00 AM|
|07/01/2014 01:00:00 PM|
|09/08/2015 05:00:00 AM|
|09/08/2015 11:41:00 AM|
|09/08/2015 12:05:00 PM|
|09/08/2015 01:10:00 PM|
|08/12/2015 05:00:00 PM|
|09/09/2015 10:30:00 AM|
|09/05/2015 10:32:00 AM|
|08/19/2018 10:38:00 AM|
|09/10/2015 11:10:00 PM|
|09/11/2015 10:35:00 AM|
|09/11/2015 04:45:00 PM|
|09/12/2015 01:45:00 PM|
|06/01/2014 12:00:00 PM|
+----------------------+
only showing top 20 rows



In [114]:
rc.select(min(col('Date')), max(col('Date'))).show()

TypeError: ignored